In [ ]:
## using pyaudio with portaudio

## https://realpython.com/playing-and-recording-sound-python/
## Playing and Recording Sound in Python

## python-sounddevice and pyaudio provide bindings for the PortAudio library for cross-platform playback
##      of WAV files.

In [ ]:
## anaconda environment setup

uses pyaudio and portaudio

		conda environment setup  -- using python 3.7.7 as thats what combined execution is using already
conda create -n ce10recsnd1 python=3.7.7
conda install pyaudio			-- installed portaudio automatically
conda install jupyter

# Check soundcard status

### Check for Soundcard with PyAudio when Soundcard is present or removed
### https://stackoverflow.com/questions/28453240/check-for-soundcard-with-pyaudio-when-soundcard-is-present-or-removed

In [4]:
import pyaudio

In [5]:
pa = pyaudio.PyAudio()
pa.get_default_output_device_info()

{'index': 22,
 'structVersion': 2,
 'name': 'default',
 'hostApi': 0,
 'maxInputChannels': 32,
 'maxOutputChannels': 32,
 'defaultLowInputLatency': 0.008684807256235827,
 'defaultLowOutputLatency': 0.008684807256235827,
 'defaultHighInputLatency': 0.034807256235827665,
 'defaultHighOutputLatency': 0.034807256235827665,
 'defaultSampleRate': 44100.0}

In [ ]:
## never worked and showed anything - even when the SETTINGS in ubuntu showed fine
"""
def get_soundcard_dev_info():
    pad_sc = pyaudio.PyAudio()
    max_devs = pad_sc.get_device_count()
    input_devices_index = []
    output_devices_index = []
    print(f"max_devs = {max_devs}")
    
    for i in range(max_devs):
        devinfo = pad_sc.get_device_info_by_index(i)
        if "TUSBAudio ASIO Driver" in devinfo['name']:
            input_devices_index.append(int(devinfo['index']))
            output_devices_index.append(int(devinfo['index']))
        if not input_devices_index:
            print(f"NONE")
        print("f{input_devices_index}")
        pad_sc.terminate()
"""
get_soundcard_dev_info()

# Play wav file with pyaudio
### pyaudio provides bindings for PortAudio, the cross-platform audio I/O library. This means that you can use pyaudio to play and record audio on a variety of platforms, including Windows, Linux, and Mac. With pyaudio, playing audio is done by writing to a .Stream

### As you may have noticed, playing sounds with pyaudio is a bit more complex than playing sounds with the libraries you’ve seen earlier. This means that it may not be your first choice if you just want to play a sound effect in your Python application. However, because pyaudio gives you more low-level control, it is possible to get and set parameters for your input and output devices, and to check your CPU load and input or output latency. It also allows you to play and record audio in callback mode, where a specified callback function is called when new data is required for playback, or available for recording. These options make pyaudio a suitable library to use if your audio needs go beyond simple playback. Now that you’ve seen how you can use a number of different libraries to play audio, it’s time to see how you can use Python to record audio yourself.

In [1]:
import pyaudio
import wave
import os

In [7]:
#playFile = r'/media/rohit/DATA/EverythingD/01SRH-BDBA Acads/Thesis/StoryGenerator/audio/wavs/st_1_HM_Rohit_file1.wav'
PLAY_FILE = r'/home/rohit/PyWDUbuntu/thesis/audio/wavs/pyaudio_rec_2.wav'

In [12]:
os.path.exists(PLAY_FILE)

True

In [20]:
if os.path.exists(PLAY_FILE):
    # Set chunk size of 1024 samples per data frame
    chunk = 1024  

    # Open the sound file 
    wf = wave.open(PLAY_FILE, 'rb')

    # Create an interface to PortAudio
    p = pyaudio.PyAudio()

    # Open a .Stream object to write the WAV file to
    # 'output = True' indicates that the sound will be played rather than recorded
    stream = p.open(format = p.get_format_from_width(wf.getsampwidth()),
                    channels = wf.getnchannels(),
                    rate = wf.getframerate(),
                    output = True)

    # Read data in chunks
    data = wf.readframes(chunk)

    # Play the sound by writing the audio data to the stream
    ###########
    ## without checking for data = b'' it never exits so changed as per this link:
    # https://stackoverflow.com/questions/34856992/when-i-play-a-wav-in-python-the-program-doesnt-stop-when-the-file-ends-leavin
    #while data != '':
    #    stream.write(data)
    #    data = wf.readframes(chunk)
    ###########
    while data != b'':
        stream.write(data)
        data = wf.readframes(chunk)
    # Close and terminate the stream
    stream.close()
    p.terminate()
else:
    print(f"File not found: {PLAY_FILE}")

print(f"\nDone")


Done


# Record Audio as wav file - fixed time
### The python-sounddevice and pyaudio libraries provide ways to record audio with Python. python-sounddevice records to NumPy arrays and pyaudio records to bytes objects. Both of these can be stored as WAV files using the scipy and wave libraries, respectively.

In [1]:
import pyaudio
import wave
import os

In [8]:
sav2File = r'/home/rohit/PyWDUbuntu/thesis/audio/wavs/pyaudio_rec_1.wav'

In [11]:
listen_for_secs = 5
chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 1 # mono
fs = 16000  # Record at 16kHz

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Recording')

## input = True means record
stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * listen_for_secs)):
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(sav2File, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

print(f"\nDone")

Recording


NameError: name 'sample_format' is not defined

# Record Audio as wav file - fixed min time and then stop at silence or max time

### Approach per: https://github.com/jeysonmc/python-google-speech-scripts/blob/master/stt_google.py   (link obtained from SO: https://stackoverflow.com/questions/19070290/pyaudio-listen-until-voice-is-detected-and-then-record-to-a-wav-file )
### The python-sounddevice and pyaudio libraries provide ways to record audio with Python. python-sounddevice records to NumPy arrays and pyaudio records to bytes objects. Both of these can be stored as WAV files using the scipy and wave libraries, respectively.

In [1]:
import pyaudio
import wave
import os
import datetime
import time

In [2]:
SAV2FILE = r'/home/rohit/PyWDUbuntu/thesis/audio/wavs/pyaudio_rec_2.wav'

In [17]:
DATA_APPEND_SECS = 2
LISTEN_MIN_SECS = 3
LISTEN_MAX_SECS = 10
MIN_VOLUME = 500

In [18]:
def listen_and_save_file(_specs_listen_list, _sav2File):
    _append_every_secs, _min_secs, _max_secs, _min_vol  = _specs_listen_list
    
    start_tick = time.time()
    
    ## setup recording parameters for deepspeech = 16kHz, mono channel, 16-bit, wav file for inference
    SAMPLE_FORMAT = pyaudio.paInt16  # 16 bits per sample
    CHANNELS = 1 # mono
    FS = 16000  # Record at 16kHz
    
    ## setup parameters for pyaudio
    CHUNCK = 1024  # Record in chunks of 1024 samples
    
    myStr = '\n'.join([
        f"\nListen parameters:",
        f"chunk = {CHUNCK}",
        f"channels = {CHANNELS}"
        f"sampling frequency = {FS} Hz",
        f"sample_format = 16 bit"
        f"append_every_secs = {_append_every_secs}",
        f"Listen Min secs = {_min_secs}",
        f"Listen Max secs = {_max_secs}",
        f"Threshold Min volume = {_min_vol}"
    ])
    print(f"{myStr}")
    myStr = None
    
    ## Initialize array to store frames
    frames = []  
    
    ## Create an interface to PortAudio
    p = pyaudio.PyAudio()
    stream = p.open(format=SAMPLE_FORMAT,
                    channels=CHANNELS,
                    rate=FS,
                    frames_per_buffer=CHUNCK,
                    input=True) ## input = True means record
    
    start_time = time.time()
    print(f"\nStarted listening....\n")
    while True:
        for i in range(0, int(FS / CHUNCK * _append_every_secs)):
            data = stream.read(CHUNCK)
            frames.append(data)
        vol = max(data)
        if (vol < _min_vol) and ( (time.time() - start_tick) > _min_secs ):
            print(f"\nStopped recording: in last {_append_every_secs} seconds, volume was below threshold")
            print(f"vol ({vol}) < threshold value ({_min_vol})")
            break
        elif (time.time() - start_tick) > _max_secs:
            print(f"\nStopped recording: reached max recording time limit = {_max_secs} seconds")
            break
    
    # Stop and close the stream 
    stream.stop_stream()
    stream.close()
    # Terminate the PortAudio interface
    p.terminate()
    
    rec_secs = time.time() - start_time
    
    print(f"\nStopped listening....\n")
    
    # Save data to WAV file
    wf = wave.open(_sav2File, 'wb')
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(p.get_sample_size(SAMPLE_FORMAT))
    wf.setframerate(FS)
    wf.writeframes(b''.join(frames))
    wf.close()
    
    print(f"\nAudio of {rec_secs:.2f} seconds saved to wav file: {_sav2File}\n")
    
    return

In [19]:
specs_listen_list = [
    DATA_APPEND_SECS,
    LISTEN_MIN_SECS,
    LISTEN_MAX_SECS,
    MIN_VOLUME 
]

listen_and_save_file(specs_listen_list, SAV2FILE)

print(f"\nDone")


Listen parameters:
chunk = 1024
channels = 1sampling frequency = 16000 Hz
sample_format = 16 bitappend_every_secs = 2
Listen Min secs = 3
Listen Max secs = 10
Threshold Min volume = 500

Started listening....


Stopped recording: in last 2 seconds, volume was below threshold
vol (255) < threshold value (500)

Stopped listening....


Audio of 3.96 seconds saved to wav file: /home/rohit/PyWDUbuntu/thesis/audio/wavs/pyaudio_rec_2.wav


Done
